In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression as lir
import pickle as pk
import scikit_posthocs as sp

In [2]:
raw_data_path = '/mnt/c/Users/Abhinav/Documents/Semester_7/BTP - 1/Avg_NL_Dist_cf_cvg.csv'
districts_path = '/mnt/c/Users/Abhinav/Documents/Semester_7/BTP - 1/Data/districts.csv'
district = "Allahabad"
years = 6
Isoutput = False

In [3]:
df = pd.read_csv(raw_data_path,header=None)
df = df.T
data = 0
def take(x):
    global data
    data = x
df.apply(lambda x: take(x) if x[74]==district else x)
data = data.reindex()
data = data[1:len(data)-3]
xaxis = np.arange(4,years*12+4)
data = np.array(data, dtype=np.float64)

In [4]:
def getSpikeDates(temp):
    arr = []
    for i in temp:
        mm = (i+3)%12 + 1
        yy = 12 + int(i/12)
        if (i%12)+3 >= 12:
            yy += 1
        arr.append(str(mm)+"-20"+str(yy))
    return arr
    
def getAnnualDates(temp, month):
    arr = []
    for i in temp:
        mm = (month+3)%12 + 1
        yy = 12 + i
        if (month%12)+3 >= 12:
            yy += 1
        arr.append(str(mm)+"-20"+str(yy))
    return arr

def intersection(a1, a2):
    a3 = []
    for x in a1:
        if x in a2:
            a3.append(x)
    return sorted(a3, key=lambda x: (int(x.split('-')[1]), int(x.split('-')[0])))

# MAD
def outliers_modified_z_score(ys, threshold):
    median_y = np.median(ys)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y
                         for y in ys]
    new1 = np.where(np.abs(modified_z_scores) > threshold)
    return new1[0]

# IQR
def outliers_iqr(noise, temp1):
    # obj = sp.outliers_gesd(noise, outliers = 20, report = True, alpha = 0.02)
    obj = sp.outliers_iqr(noise, coef=temp1) #return_ = 'outliers', 
    z = 0
    arr = []
    for i in range(len(noise)):
        if obj[z] == noise[i]:
            z += 1
        else:
            arr.append(i)
    return arr

# Z_SCORE
def outliers_z_score(noise, influence, threshold):
    values = noise
    anomaly = np.zeros((12,years))
    curmean = 0.0
    curdev = 0.0
    newel = 0.0
    mean_for = []
    mean_back = []
    std_for = []
    std_back = []
    thresh = []
    abs_val = []
    curlen = 1
    win = 12

    mean_for.append(values[0])
    std_for.append(0.0)
    for i in range(1,len(values)):
        if abs(values[i]-curmean)>curdev*threshold:
            newel = influence*values[i]+(1-influence)*values[(i-1)]
        else:
            newel = values[i]
        sum1 = newel
        for j in range(max(0,i-win+1),i):
            sum1 += values[j]
        temp_mean = sum1/(min(win, i+1))
        sum1 = (newel - temp_mean)**2
        for j in range(max(0,i-win+1),i):
            sum1 += (values[j] - temp_mean)**2
        temp_std = math.sqrt(sum1/(min(win, i+1)))
        curmean = temp_mean
        curdev = temp_std
        mean_for.append(temp_mean)
        std_for.append(temp_std)

    mean_back.append(values[len(values)-1])
    std_back.append(0.0)
    curmean=0.0
    curdev=0.0
    for i in range(len(values)-2,-1,-1):
        if abs(values[i]-curmean)>curdev*threshold:
            newel = influence*values[i]+(1-influence)*values[(i+1)]
        else:
            newel = values[i]
        sum1 = newel
        for j in range(i+1,min(len(values),i+win)):
            sum1 += values[j]
        temp_mean = sum1/(min(win, len(values)-i))
        sum1 = (newel - temp_mean)**2
        for j in range(i+1,min(len(values),i+win)):
            sum1 += (values[j] - temp_mean)**2
        temp_std = math.sqrt(sum1/(min(win, len(values)-i)))
        curmean = temp_mean
        curdev = temp_std
        mean_back.append(temp_mean)
        std_back.append(temp_std)

    avg_mean = [mean_for[i]/2 + mean_back[len(values)-1-i]/2 for i in range(0,len(values))]
    avg_std = [std_for[i]/2 + std_back[len(values)-1-i]/2 for i in range(0,len(values))]

    for i in range(0,len(values)):
        thresh.append(avg_std[i]*threshold)
        abs_val.append(abs(values[i]-avg_mean[i]))
        if abs(values[i]-avg_mean[i])>avg_std[i]*threshold:
            anomaly[int(i%12)][int(i/12)]=1
    
    arr_res = []
    if(Isoutput):
        plt.figure(figsize=(10,12))
    for i in range(years):
        for j in range(12):
            if anomaly[j][i] == 1:
                tt = i*12 + j
                arr_res.append(tt)
                if(Isoutput):
                    plt.plot(tt+1, abs_val[tt], 'o', color='black')

    xaxis = np.arange(1,years*12+1)
    if(Isoutput):
        plt.plot(xaxis,list((thresh)))
        plt.plot(xaxis,list((abs_val)))
        for i in range(1,4):
            plt.axhline(y=i*np.mean(abs_val), color='r', linestyle='-')
    
    return arr_res

def find_anomalies_spike(noise, maxa, b1, b2, b3):
    global influence
    error = 0.001
    temp = []
    finalres = ["$"]

    if b1 > 0:
        low = 0.001
        high = 20.0
        while (high - low) > error:
            mid = (low+high)/2
            temp = outliers_modified_z_score(noise, mid)
            if (len(temp)) > maxa:
                low = mid
            else:
                high = mid
        finalres = getSpikeDates(temp)
    
    if b2 > 0:
        low = 0.001
        high = 20.0
        while (high - low) > error:
            mid = (low+high)/2
            temp = outliers_iqr(noise, mid)
            if (len(temp)) > maxa:
                low = mid
            else:
                high = mid
        if len(finalres) > 0 and finalres[0] == "$":
            finalres = getSpikeDates(temp)
        else:
            finalres = intersection(finalres, getSpikeDates(temp))
    
    if b3 > 0:
        low = 0.001
        high = 20.0
        while (high - low) > error:
            mid = (low+high)/2
            temp = outliers_z_score(noise, influence, mid)
            if (len(temp)) > maxa:
                low = mid
            else:
                high = mid
        if len(finalres) > 0 and finalres[0] == "$":
            finalres = getSpikeDates(temp)
        else:
            finalres = intersection(finalres, getSpikeDates(temp))
    
    if len(finalres) > 0 and finalres[0] == "$":
            finalres = []
    return finalres

def find_anomalies_annual(noise, maxa, b1, b2, b3):
    global influence1
    error = 0.001
    temp = []
    finalres = ["$"]

    if b1 > 0:
        low = 0.001
        high = 20.0
        while (high - low) > error:
            mid = (low+high)/2
            temp = []
            for ii in range(12):
                tempnoise = [noise[ii + j*12] for j in range(years)]
                temp += getAnnualDates(outliers_modified_z_score(tempnoise, mid), ii)
            if (len(temp)) > maxa:
                low = mid
            else:
                high = mid
        finalres = temp
    
    if b2 > 0:
        low = 0.001
        high = 20.0
        temp = []
        while (high - low) > error:
            mid = (low+high)/2
            temp = []
            for ii in range(12):
                tempnoise = [noise[ii + j*12] for j in range(years)]
                temp += getAnnualDates(outliers_iqr(tempnoise, mid), ii)
            if (len(temp)) > maxa:
                low = mid
            else:
                high = mid
        if len(finalres) > 0 and finalres[0] == "$":
            finalres = temp
        else:
            finalres = intersection(finalres, temp)
    
    if b3 > 0:
        low = 0.001
        high = 20.0
        while (high - low) > error:
            mid = (low+high)/2
            temp = outliers_z_score(noise, influence1, mid)
            if (len(temp)) > maxa:
                low = mid
            else:
                high = mid
        if len(finalres) > 0 and finalres[0] == "$":
            finalres = temp
        else:
            finalres = intersection(finalres, temp)
    
    if len(finalres) > 0 and finalres[0] == "$":
            finalres = []
    return finalres

## Decomposition

In [5]:
# Trend
xaxis = np.arange(4,years*12+4)
yaxis = np.array(data[0:].tolist(),dtype=np.float32)

# Detrend 1 use regression line method
X = [i for i in range(0, len(yaxis))]
X = np.reshape(X, (len(X), 1))
y = yaxis
model = lir()
model.fit(X, y)
# calculate trend
yaxis1 = model.predict(X)

# Detrend 2
yaxis2 = [yaxis[i]-yaxis1[i] for i in range(0,len(yaxis))]

# Detrend 3
yaxis3=[]
for i in range(len(yaxis2)):
    sum1=0
    for j in range(years):
        sum1+=yaxis2[i%12+12*j]
    yaxis3.append(sum1/years)

noise = [yaxis[i]-yaxis1[i]-yaxis3[i] for i in range(len(yaxis))]

if(Isoutput):
    plt.figure(figsize=(15,10))
    plt.plot(xaxis,data,marker='o', color='b')
    plt.plot(xaxis,np.array(yaxis3)+np.array(yaxis1), color='r', linestyle='--')
    # plt.plot(xaxis,np.array(yaxis31), color='g', linestyle='--')
    for j in range(int((years*12)/5)):
        plt.axvline(x=j*5,linestyle='--')
    plt.show()
    plt.figure(figsize=(15,10))
    plt.plot(0,np.min(np.array(yaxis)))
    plt.plot(0,np.max(np.array(yaxis)))
    plt.plot(xaxis,noise,marker='o', color='b')
    plt.axhline(y=0,linestyle='--',color='r')
    for j in range(int((years*12)/5)):
        plt.axvline(x=j*5,linestyle='--')

noise = np.array(noise)
noise = (noise - noise.mean(axis=0))/(noise.std(axis=0))

## Spike Anomaly

In [14]:
max_anomalies = 13
influence = 0.2

tset = set()
res = []
for x in range(1, max_anomalies+1):
    temp = find_anomalies_spike(noise, x, 1, 1, 1)
    tt = []
    for y in temp:
        if y not in tset:
            tset.add(y)
            tt.append(y)
    if len(tt) > 0:
        res.append(tt)

z=1
for x in res:
    for y in x:
        ss = 'Rank ' + str(z) + ': ' + y
        z += 1
        print(ss)

Rank 1: 1-2015
Rank 2: 10-2013
Rank 3: 9-2014
Rank 4: 12-2016
Rank 5: 1-2013
Rank 6: 9-2016
Rank 7: 6-2013
Rank 8: 12-2013
Rank 9: 6-2012
Rank 10: 4-2014
Rank 11: 6-2016


## Annual Anomaly

In [7]:
max_anomalies1 = 8
influence1 = 0.2

tset = set()
res1 = []
for x in range(1, max_anomalies1+1):
    temp = find_anomalies_annual(noise, x, 1, 1, 0)
    tt = []
    for y in temp:
        if y not in tset:
            tset.add(y)
            tt.append(y)
    if len(tt) > 0:
        res1.append(tt)

z=1
for x in res1:
    for y in x:
        ss = 'Rank ' + str(z) + ': ' + y
        z += 1
        print(ss)

Rank 1: 4-2014
Rank 2: 10-2013
Rank 3: 9-2014
Rank 4: 1-2015
Rank 5: 4-2016
Rank 6: 2-2014
Rank 7: 3-2013
Rank 8: 11-2015
